In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from google.colab import drive
     

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 1s (5,163 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fro

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
from google.colab import drive

In [7]:
source = '/content/drive/MyDrive/CAPSTONE PROJECT BANGKIT/skincare product dataset'
#path into train and test images
train_images = os.path.join(source, 'images')
train_annotations = os.path.join(source, 'mask')

In [8]:
print("Contents of base directory:")
print(os.listdir(source))

print("\nContents of train directory:")
print(os.listdir(f'{source}/train_images'))

print("\nContents of validation directory:")
print(os.listdir(f'{source}/train_annotations'))

Contents of base directory:
['train_images', 'train_annotations', 'images', 'mask']

Contents of train directory:
['clear.jpeg', 'photo1684971609.jpeg', 'photo1684971618.jpeg', 'photo1684971633.jpeg', 'photo1684971642.jpeg', 'photo1684971652.jpeg', 'photo1684971668.jpeg', 'photo1684971800.jpeg']

Contents of validation directory:
['clear.jpeg', 'photo1684971609 - Copy.jpeg', 'photo1684971618 - Copy.jpeg', 'photo1684971633 - Copy.jpeg', 'photo1684971642 - Copy.jpeg', 'photo1684971668 - Copy.jpeg', 'photo1684971800 - Copy.jpeg']


In [9]:
sorted_image = sorted(os.listdir(train_images))
sorted_mask = sorted(os.listdir(train_annotations))

In [10]:
img_size = 128

In [12]:
def get_segmented_img(img):
    # Membuat array kosong dengan ukuran (img_size, img_size, 1) untuk menyimpan label-label segmentasi.
    labels = np.zeros((img_size, img_size, 1))
    
    # Mengubah ukuran gambar menjadi (img_size, img_size).
    img = cv2.resize(img, (img_size, img_size))
    
    # Mengambil saluran pertama dari gambar.
    img = img[:, :, 0]
    
    # Mengisi array labels dengan nilai 0 atau 1 berdasarkan piksel gambar.
    # Jika nilai piksel tidak sama dengan 0, dianggap sebagai bagian tersegmentasi dan diberi nilai 1.
    # Jika nilai piksel sama dengan 0, dianggap sebagai bagian tidak tersegmentasi dan diberi nilai 0.
    labels[:, :, 0] = (img != 0).astype(int)

    # Mengembalikan array labels sebagai hasil dari fungsi.
    return labels

In [13]:
x = []
y = []

# Iterasi melalui setiap indeks dalam daftar file gambar di direktori train_images.
for index in range(len(os.listdir(train_images))):
    # Membaca gambar menggunakan OpenCV dari path yang diberikan oleh os.path.join.
    img = cv2.imread(os.path.join(train_images, sorted_image[index]))
    
    # Mengubah gambar menjadi skala abu-abu.
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Melakukan thresholding pada gambar menggunakan nilai ambang 150.
    # Piksel dengan intensitas di atas ambang akan menjadi putih, sedangkan yang di bawah ambang akan menjadi hitam.
    ret, img = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY_INV)
    
    # Mengubah ukuran gambar menjadi ukuran (img_size, img_size).
    img = cv2.resize(img, (img_size, img_size))
    
    # Menambahkan dimensi tambahan pada gambar menggunakan np.expand_dims.
    # Dalam konteks ini, dimensi tambahan ditambahkan di akhir untuk sesuai dengan format data model yang diharapkan.
    img = np.expand_dims(img, axis=-1)
    
    # Normalisasi gambar dengan membaginya dengan 255.
    img = img / 255
    
    # Membaca masker gambar yang sesuai dengan gambar yang sedang diproses.
    annotation = cv2.imread(os.path.join(train_annotations, sorted_mask[index]))
    
    # Mengubah masker menjadi skala abu-abu.
    annotation = cv2.cvtColor(annotation, cv2.COLOR_BGR2GRAY)
    
    # Mengubah ukuran masker menjadi ukuran (img_size, img_size).
    annotation = cv2.resize(annotation, (img_size, img_size))
    
    # Membuat saluran warna untuk masker dengan mengulang saluran abu-abu tiga kali.
    annotation = np.stack((annotation,)*3, axis=-1)
    
    # Menggunakan fungsi get_segmented_img untuk menghasilkan label segmentasi dari masker.
    annotation = get_segmented_img(annotation)
    
    # Menambahkan gambar dan label segmentasi ke dalam daftar x dan y, masing-masing.
    x.append(img)
    y.append(annotation)

In [14]:
# Split Dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, random_state = 1)

x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [15]:
print(x_train.shape)
print(x_test.shape)

(17, 128, 128, 1)
(5, 128, 128, 1)


In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam

inputs = Input((img_size, img_size, 1))

conv1 = Conv2D(64, 3, activation= 'relu', kernel_initializer='he_normal',padding = 'same')(inputs)
conv1 = Conv2D(64, 3, activation= 'relu',kernel_initializer='he_normal', padding = 'same')(conv1)
pool1 = MaxPooling2D(2,2)(conv1)

conv2 = Conv2D(128, 3, activation= 'relu', kernel_initializer='he_normal', padding = 'same')(pool1)
conv2 = Conv2D(128, 3, activation= 'relu', kernel_initializer='he_normal',padding = 'same')(conv2)
pool2 = MaxPooling2D(2,2)(conv2)

conv3 = Conv2D(256, 3, activation= 'relu', kernel_initializer='he_normal',padding = 'same')(pool2)
conv3 = Conv2D(256, 3, activation= 'relu',kernel_initializer='he_normal', padding = 'same')(conv3)
pool3 = MaxPooling2D(2,2)(conv3)

conv4 = Conv2D(512, 3, activation= 'relu', kernel_initializer='he_normal',padding = 'same')(pool3)
conv4 = Conv2D(512, 3, activation= 'relu',kernel_initializer='he_normal', padding = 'same')(conv4)
pool4 = MaxPooling2D(2,2)(conv4)

conv5 = Conv2D(1024, 3, activation= 'relu',kernel_initializer='he_normal', padding = 'same')(pool4)
conv5 = Conv2D(1024, 3, activation= 'relu', kernel_initializer='he_normal',padding = 'same')(conv5)
dropout5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation = 'relu', kernel_initializer='he_normal',padding = 'same')(UpSampling2D(size = (2,2))(dropout5))
merge6 = concatenate([conv4,up6], axis = 3)
conv6 = Conv2D(512, 3, activation = 'relu', kernel_initializer='he_normal',padding = 'same')(merge6)
conv6 = Conv2D(512, 3, activation = 'relu', kernel_initializer='he_normal',padding = 'same')(conv6)

up7 = Conv2D(256, 2, activation = 'relu', kernel_initializer='he_normal',padding = 'same')(UpSampling2D(size = (2,2))(conv6))
merge7 = concatenate([conv3,up7], axis = 3)
conv7 = Conv2D(256, 3, activation = 'relu', kernel_initializer='he_normal', padding = 'same')(merge7)
conv7 = Conv2D(256, 3, activation = 'relu',kernel_initializer='he_normal', padding = 'same')(conv7)

up8 = Conv2D(128, 2, activation = 'relu', kernel_initializer='he_normal',padding = 'same')(UpSampling2D(size = (2,2))(conv7))
merge8 = concatenate([conv2,up8], axis = 3)
conv8 = Conv2D(128, 3, activation = 'relu', kernel_initializer='he_normal',padding = 'same')(merge8)
conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same')(conv8)
,
up9 = Conv2D(64, 2, activation = 'relu', kernel_initializer='he_normal', padding = 'same')(UpSampling2D(size = (2,2))(conv8))
merge9 = concatenate([conv1,up9], axis = 3)
conv9 = Conv2D(64, 3, activation = 'relu',kernel_initializer='he_normal', padding = 'same')(merge9)
conv9 = Conv2D(64, 3, activation = 'relu',kernel_initializer='he_normal', padding = 'same')(conv9)

conv9 = Conv2D(2, 3, activation = 'relu', kernel_initializer='he_normal',padding = 'same')(conv9)
conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
     
model = Model(inputs=inputs, outputs=conv9)

model.compile(optimizer = Adam(learning_rate = 1e-4),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
    
model.summary()
     

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

In [17]:
model.fit(x_train, y_train,
          epochs=10,
          steps_per_epoch=500,
          validation_data=(x_test, y_test),
          validation_steps=300,
          shuffle=True)

Epoch 1/10
170/500 [=========>....................] - ETA: 14s - loss: 0.2781 - accuracy: 0.9046

500/500 [==============================] - 30s 16ms/step - loss: 0.2781 - accuracy: 0.9046 - val_loss: 0.0000e+00 - val_accuracy: 0.9691


In [18]:
model.save('bounding_box.h5')

In [19]:
from google.colab import files
files.download('bounding_box.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>